In [107]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [108]:
tag = 'christmasbreak'
browser = webdriver.Chrome('/Users/matt/Downloads/chromedriver')
browser.get('https://twitter.com/hashtag/' + tag + '?lang=en')

In [109]:
links=[]

def scrape(start, end):
    print('scrolling: {}-{}'.format(start, end))
    Pagelength = browser.execute_script("window.scrollTo({}, {});".format(start, end))
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.findAll('div', {'data-permalink-path': True})
    for div in script:
        status = div.get('data-permalink-path')
        links.append('https://www.twitter.com{}'.format(status))
    
    time.sleep(5)
    if (end < 50000):
        scrape(start + 3000, end + 3000)

In [110]:
scrape(0, 6000)

scrolling: 0-6000
scrolling: 3000-9000
scrolling: 6000-12000
scrolling: 9000-15000
scrolling: 12000-18000
scrolling: 15000-21000
scrolling: 18000-24000
scrolling: 21000-27000
scrolling: 24000-30000
scrolling: 27000-33000
scrolling: 30000-36000
scrolling: 33000-39000
scrolling: 36000-42000
scrolling: 39000-45000
scrolling: 42000-48000
scrolling: 45000-51000


In [111]:
print('Found {} tweets'.format(len(links)))

Found 320 tweets


In [112]:
result = pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        user = body.find('b', {'class': 'u-linkComplex-target'})
        location = body.find('span', {'class': 'ProfileHeaderCard-locationText u-dir'})
        content = body.find('p', {'class': 'tweet-text'})
        tweetDetails = body.find('div', {'class': 'tweet-details-fixer'})
        timeStamp = tweetDetails.find('span').find('span').text
        tweet = {
            'user': '@{}'.format(user.text),
            'location': location.text.strip(),
            'tweet': content.text,
            'timeStamp': timeStamp
        }
        x = pd.DataFrame.from_dict(json_normalize(tweet), orient='columns') 
        x = x[['user', 'location', 'tweet', 'timeStamp']]
        result = result.append(x) 
    except:
        np.nan

result.index = range(len(result.index))

In [113]:
result = result.drop_duplicates()
print('scraped {} tweets'.format(len(result)))
result.head()

scraped 20 tweets


,user,location,tweet,timeStamp
0,@BridportMan,Bridport,We'll be on Christmas break from December 16th...,5:54 AM - 27 Nov 2019
1,@HILeicesterCity,"Leicester, England",Plan your pre-Christmas break to Leicester now...,8:08 AM - 27 Nov 2019
2,@afaranwide,,"Sunset over Phu Quoc, Vietnam.\n\nLook out for...",2:41 AM - 27 Nov 2019
3,@Sarah___Mary,South Wales,When you realise you only have to survive one ...,11:21 PM - 26 Nov 2019
4,@EduClickAfrica,"Centre, Cameroon",Our #Techlab offers affordable #excursion expe...,12:05 AM - 25 Nov 2019
